In [0]:
import pyspark.sql

input_path="s3://ayush-varma/Input/StreamingData/"
checkpoint_path="s3://ayush-varma/output/checkpoint/"


raw_df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", checkpoint_path)
    .option("inferSchema", 'True')
    .option("cloudFiles.schemaEvolutionMode","rescue" )
    .load(input_path)
)

display(raw_df)

In [0]:
from pyspark.sql.functions import input_file_name, count

# inserting a new column displaying the source file name
files_df= (
    raw_df.withColumn("file_source_name", input_file_name())
)

display(files_df)

In [0]:
files_df.writeStream \
    .format('delta') \
    .option('checkpointLocation', checkpoint_path) \
    .option("mergeschema", "true") \
    .queryName('employee-delta') \
    .trigger(processingTime="2 seconds") \
    .table('employee')